<a href="https://colab.research.google.com/github/uno110/sample/blob/master/yahoo_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

#seleniumとchromedriverをインストールしてヘッドレスクロームを準備する
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import sys
import csv
from time import sleep
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; Touch; .NET4.0E; .NET4.0C; .NET CLR 3.5.30729; .NET CLR 2.0.50727; .NET CLR 3.0.30729; Tablet PC 2.0; rv:11.0) like Gecko')
driver = webdriver.Chrome('chromedriver',options=options)

try:
  result=[]
  for i in range(1,39):
    url="https://soccer.yahoo.co.jp/ws/schedule/52?class={}".format(i)
    driver.get(url)
    WebDriverWait(driver, 10).until(expected_conditions.visibility_of_element_located((By.CLASS_NAME, 'sn-table')))
    div = driver.find_element(By.CLASS_NAME, 'sn-table')
    tr_elems= div.find_elements(By.TAG_NAME, 'tr')
    tr_elems.pop(0)
    
    for tr in tr_elems:
      game=[]
      game_elems=[]
      for td in tr.find_elements(By.TAG_NAME, 'td'):
        game.append(td.text)
      time=game[0].split("\n")
      status=game[3].split("\n")
      score=status[0].split("-")
      game_elems.append("{}節".format(i))
      game_elems.append(time[0])
      game_elems.append(time[1])
      game_elems.append(game[1])
      game_elems.append(game[5])
      game_elems.append(status[1])
      game_elems.append(score[0])
      game_elems.append(score[1])
      result.append(game_elems)
finally:
  driver.quit()








 






ModuleNotFoundError: ignored

In [ ]:
import csv
header=['節数', '試合日', '開始時刻', 'ホームチーム', 'アウェイチーム', '試合状況', 'ホームチームスコア', 'アウェイチームスコア']
with open('result.csv', 'w')as f:
  writer = csv.writer(f)  # writerオブジェクトを作成
  writer.writerow(header)
  writer.writerows(result) 

In [ ]:
import openpyxl
wb = openpyxl.Workbook()
ws = wb.worksheets[0]
file_name = "結果.xlsx"
header=['節数', '試合日', '開始時刻', 'ホームチーム', 'アウェイチーム', '試合状況', 'ホームチームスコア', 'アウェイチームスコア']
ws = wb.active
def write_data(sheet, list, start_row, start_col):
  for y, row in enumerate(data):
    for x, cell in enumerate(row):
      sheet.cell(row=y+start_row,
                column=x+start_col,
                value=data[y][x])
data=[]
data.append(header) 
data.extend(result)
write_data(ws, data,1,1)
#名前を付けて保存
wb.save('結果.xlsx')

In [ ]:
!pip install gspread
!pip install oauth2client

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials 
from google.colab import drive
drive.mount('/content/drive')

scope=['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
c = ServiceAccountCredentials.from_json_keyfile_name('drive/My Drive/my-project-test-306406-cd1d807083f0.json', scope)

gc = gspread.authorize(c)
workbook=gc.open_by_key('1daijg_647OO9hjY_Pvzrt1AGquqDeo_7sVyg-qrordw')
worksheet = workbook.add_worksheet(title="", rows=381,  cols=8)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
header=['節数', '試合日', '開始時刻', 'ホームチーム', 'アウェイチーム', '試合状況', 'ホームチームスコア', 'アウェイチームスコア']
data=[]
data.append(header) 
data.extend(result)
new_data=sum(data,[])

cell_list=worksheet.range('A1:H379')
for i , cell in enumerate(cell_list):
  cell.value=new_data[i]
worksheet.update_cells(cell_list)

{'spreadsheetId': '1daijg_647OO9hjY_Pvzrt1AGquqDeo_7sVyg-qrordw',
 'updatedCells': 3032,
 'updatedColumns': 8,
 'updatedRange': "'シート29'!A1:H379",
 'updatedRows': 379}